In [1]:

import numpy as np
import matplotlib.pyplot as plt
import gpflow as gpf
import tensorflow as tf

from gpflow.utilities import print_summary
from gpflow.ci_utils import ci_niter
from gpflow.optimizers import NaturalGradient
from gpflow import set_trainable

gpf.config.set_default_float(np.float64)
gpf.config.set_default_summary_fmt("notebook")
np.random.seed(0)
%matplotlib inline

MAXITER = ci_niter(2000)

In [2]:

N = 100  # number of points
D = 1  # number of input dimensions
M = 15  # number of inducing points
L = 2  # number of latent GPs
P = 3  # number of observations = output dimensions

In [3]:
def generate_data(N=100):
    X = np.random.rand(N)[:, None] * 10 - 5  # Inputs = N x D
    G = np.hstack((0.5 * np.sin(3 * X) + X, 3.0 * np.cos(X) - X))  # G = N x L
    W = np.array([[0.5, -0.3, 1.5], [-0.4, 0.43, 0.0]])  # L x P
    F = np.matmul(G, W)  # N x P
    Y = F + np.random.randn(*F.shape) * [0.2, 0.2, 0.2]

    return X, Y

In [4]:
X, Y = data = generate_data(N)

In [5]:

# create multi-output kernel
kernel = gpf.kernels.SharedIndependent(
    gpf.kernels.SquaredExponential() + gpf.kernels.Linear(), output_dim=P
)

kernel = gpf.kernels.SquaredExponential() + gpf.kernels.Linear()

In [6]:
tf.eye(X.shape[0], dtype="float") 

<tf.Tensor: shape=(100, 100), dtype=float32, numpy=
array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)>

In [8]:
kernel(X)

InternalError: Blas GEMM launch failed : a.shape=(100, 1), b.shape=(100, 1), m=100, n=100, k=1 [Op:MatMul]

In [11]:
# create SVGP model as usual and optimize
m = gpf.models.VGP(data=(X, Y), kernel=kernel, likelihood=gpf.likelihoods.Gaussian(), num_latent_gps=2)
print_summary(m)

name,class,transform,prior,trainable,shape,dtype,value
VGP.kernel.kernel.kernels[0].variance,Parameter,Softplus,,True,(),float64,1.0
VGP.kernel.kernel.kernels[0].lengthscales,Parameter,Softplus,,True,(),float64,1.0
VGP.kernel.kernel.kernels[1].variance,Parameter,Softplus,,True,(),float64,1.0
VGP.likelihood.variance,Parameter,Softplus + Shift,,True,(),float64,1.0
VGP.q_mu,Parameter,Identity,,True,"(100, 2)",float64,"[[0., 0...."
VGP.q_sqrt,Parameter,FillTriangular,,True,"(2, 100, 100)",float64,"[[[1., 0., 0...."


In [ ]:
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [12]:
natgrad = NaturalGradient(gamma=1.0)
adam = tf.optimizers.Adam(0.01)

set_trainable(m.q_mu, False)
set_trainable(m.q_sqrt, False)

logf = []

for _ in range(ci_niter(1000)):
    adam.minimize(m.training_loss, m.trainable_variables)
    natgrad.minimize(m.training_loss, [(m.q_mu, m.q_sqrt)])   
    logf.append(m.elbo())

InvalidArgumentError: Incompatible shapes: [100,3,3] vs. [100,100] [Op:AddV2]